In [ ]:
import tensorflow as tf

opt = {
    "kernel_initializer": "xavier",
    "bias_initializer": "zeros",
    "batch_normalization": True,
    "dropout": True,
    "dropout_prob_keep": 0.4,
    "depth": 4,
    "filters": [256, 512, 1024, 512],
    "l2_regularizer_weight": 1e-12,
    'batch_size': 64,
    'lr': 1e-3,
}

In [ ]:
class ResidualBlock(tf.keras.layers.Layer):
    def __init__(self, filters,
                 kernel_initializer=tf.keras.initializers.GlorotNormal(),
                 bias_initializer=tf.keras.initializers.zeros(),
                 regularizer=None,
                 batch_norm=False,
                 name=''):
        super(ResidualBlock, self).__init__(name=name)

        self.conv2a = tf.keras.layers.Conv2D(filters, (1, 1),
                                             activation=None,
                                             use_bias=True,
                                             kernel_initializer=kernel_initializer,
                                             bias_initializer=bias_initializer,
                                             kernel_regularizer=regularizer,
                                             bias_regularizer=regularizer
                                             )
        self.conv2d = tf.keras.layers.Conv2D(filters, (1, 1),
                                             activation=None,
                                             use_bias=True,
                                             kernel_initializer=kernel_initializer,
                                             bias_initializer=bias_initializer,
                                             kernel_regularizer=regularizer,
                                             bias_regularizer=regularizer
                                             )

        self.batch_norm = batch_norm
        if self.batch_norm:
            self.bn2a = tf.keras.layers.BatchNormalization()
            self.bn2b = tf.keras.layers.BatchNormalization()
            self.bn2c = tf.keras.layers.BatchNormalization()
            self.bn2d = tf.keras.layers.BatchNormalization()

        self.conv2b = tf.keras.layers.Conv2D(filters, (3, 3), padding='valid',
                                             activation=None,
                                             use_bias=True,
                                             kernel_initializer=kernel_initializer,
                                             bias_initializer=bias_initializer,
                                             kernel_regularizer=regularizer,
                                             bias_regularizer=regularizer
                                             )

        self.conv2c = tf.keras.layers.Conv2D(filters, (1, 1),
                                             activation=None,
                                             use_bias=True,
                                             kernel_initializer=kernel_initializer,
                                             bias_initializer=bias_initializer,
                                             kernel_regularizer=regularizer,
                                             bias_regularizer=regularizer
                                             )
        self.conv2d = tf.keras.layers.Conv2D(filters, (3, 3), padding='valid',
                                             activation=None,
                                             use_bias=True,
                                             kernel_initializer=kernel_initializer,
                                             bias_initializer=bias_initializer,
                                             kernel_regularizer=regularizer,
                                             bias_regularizer=regularizer
                                             )

    def call(self, input_tensor, training=False):
        x = self.conv2a(input_tensor)
        if self.batch_norm:
            x = self.bn2a(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv2b(x)
        if self.batch_norm:
            x = self.bn2b(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv2c(x)
        input_tensor = self.conv2d(input_tensor)
        if self.batch_norm:
            x = self.bn2c(x, training=training)
            input_tensor = self.bn2d(input_tensor, training=training)
        x += input_tensor
        return tf.nn.relu(x)

In [ ]:
class CNNModel(tf.keras.Model):
    def __init__(self, opt):
        super(CNNModel, self).__init__(name='')
        self.depth = opt['depth']

        if opt['kernel_initializer'] == 'gaussian':
            kernel_initializer = tf.keras.initializers.RandomNormal(mean=0., stddev=1.)
        elif opt['kernel_initializer'] == 'xavier' or opt['kernel_initializer'] == 'glorot':
            kernel_initializer = tf.keras.initializers.GlorotNormal()
        elif opt['kernel_initializer'] == 'zeros':
            kernel_initializer = tf.keras.initializers.zeros()
        else:
            kernel_initializer = tf.keras.initializers.GlorotNormal()

        if opt['bias_initializer'] == 'gaussian':
            bias_initializer = tf.keras.initializers.RandomNormal(mean=0., stddev=1.)
        elif opt['kernel_initializer'] == 'xavier' or opt['kernel_initializer'] == 'glorot':
            bias_initializer = tf.keras.initializers.GlorotNormal()
        elif opt['kernel_initializer'] == 'zeros':
            bias_initializer = tf.keras.initializers.zeros()
        else:
            bias_initializer = tf.keras.initializers.GlorotNormal()

        if 'l2_regularizer_weight' in opt:
            regularizer = tf.keras.regularizers.L2(opt['l2_regularizer_weight'])
        else:
            regularizer = None

        self.batch_norm = opt['batch_normalization']
        filters = opt['filters']
        self.pooling = tf.keras.layers.MaxPool2D(
            pool_size=(3, 3), strides=2, padding='valid'
        )
        for i in range(self.depth):
            name = "resblock{}".format(i + 1)
            setattr(self, name, ResidualBlock(filters[i],
                                              kernel_initializer=kernel_initializer,
                                              bias_initializer=bias_initializer,
                                              regularizer=regularizer,
                                              batch_norm=self.batch_norm,
                                              name=name)
                    )
            """
            name = "conv{}".format(i + 1)
            setattr(self, name, tf.keras.layers.Conv2D(filters[i], (3, 3),
                                                       activation=None,
                                                       use_bias=True,
                                                       kernel_initializer=kernel_initializer,
                                                       bias_initializer=bias_initializer,
                                                       kernel_regularizer=regularizer,
                                                       bias_regularizer=regularizer
                                                       )
                    )
            """

        self.fc1 = tf.keras.layers.Dense(
            256,
            activation=None,
            use_bias=True,
            kernel_initializer=kernel_initializer,
            bias_initializer=bias_initializer,
            kernel_regularizer=regularizer,
            bias_regularizer=regularizer
        )
        self.fc2 = tf.keras.layers.Dense(
            128,
            activation=None,
            use_bias=True,
            kernel_initializer=kernel_initializer,
            bias_initializer=bias_initializer,
            kernel_regularizer=regularizer,
            bias_regularizer=regularizer
        )
        if opt['dropout']:
            self.dropout = tf.keras.layers.Dropout(opt['dropout_prob_keep'])
        else:
            self.dropout = None
        self.reg = tf.keras.layers.Dense(
            1,
            activation=None,
            use_bias=True,
            kernel_initializer=kernel_initializer,
            bias_initializer=bias_initializer,
            kernel_regularizer=regularizer,
            bias_regularizer=regularizer
        )
        self.flatten = tf.keras.layers.Flatten()

    def call(self, x, training=False):
        for i in range(self.depth):
            """
            name = "conv{}".format(i + 1)
            f = getattr(self, name)
            x = f(x)
            
            # TODO: We may add if condition for batch normalization
            x = tf.nn.relu(x)
            """
            name = "resblock{}".format(i + 1)
            f = getattr(self, name)
            x = f(x, training=training)
            x = self.pooling(x)

        # x = tf.reshape(x, [-1, 1 * 1 * 256])
        x = self.flatten(x)
        # TODO: We may add if condition for batch normalization
        x = self.fc1(x)
        x = tf.nn.relu(x)
        x = self.fc2(x)
        x = tf.nn.relu(x)
        if self.dropout is not None:
            x = self.dropout(x, training=training)
        x = self.reg(x)
        return x, tf.math.round(x)


In [ ]:
model = CNNModel(opt)
#_ = model(tf.zeros([1, 91, 91, 3]))

optimizer = tf.keras.optimizers.Adam(learning_rate=opt['lr'])
mae_loss_fn = tf.keras.losses.MeanAbsoluteError()


In [ ]:
# model.build([None,91,91,1])

In [ ]:
# model.summary()

In [ ]:
%cd /content

/content


In [ ]:
%cd /content/drive/MyDrive/CS559/Homework

/content/drive/MyDrive/CS559/Homework


In [ ]:
import numpy as np
x_train = np.load("training_data.npy")
x_train = np.expand_dims(x_train, axis=-1)
y_train = np.load("training_labels.npy")
x_val = np.load("validation_data.npy")
x_val = np.expand_dims(x_val, axis=-1)

y_val = np.load("validation_labels.npy")
x_test = np.load("test_data.npy")
x_test = np.expand_dims(x_test, axis=-1)
y_test = np.load("test_labels.npy")

x_train = x_train / 255
x_val = x_val / 255
x_test = x_test / 255


In [ ]:
# train_dataset = tf.data.Dataset.from_tensor_slices(x_train)
batch_size = opt['batch_size']
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

# Prepare the validation dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

In [ ]:
import time


train_loss_metric = tf.keras.metrics.MeanAbsoluteError()
val_loss_metric = tf.keras.metrics.MeanAbsoluteError()

train_acc_metric = tf.keras.metrics.Accuracy(name="train_acc_metric")
val_acc_metric = tf.keras.metrics.Accuracy(name="val_acc_metric")



In [ ]:
epochs = 50
train_loss_results = []
train_accuracy_results = []
val_loss_results = []
val_accuracy_results = []

for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.device('/gpu:0') as dev:
            with tf.GradientTape() as tape:
                output, pred = model(x_batch_train, training=True)
                loss_value = mae_loss_fn(y_batch_train, output)
            grads = tape.gradient(loss_value, model.trainable_weights)
            optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Update training metric.
        train_loss_metric.update_state(y_batch_train, output)
        train_acc_metric.update_state(y_batch_train, pred)
        """
        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %d samples" % ((step + 1) * batch_size))
        """
    # Display metrics at the end of each epoch.
    train_loss = train_loss_metric.result()
    train_acc = train_acc_metric.result()
    print("Training loss over epoch: %.4f, acc %.4f" % (float(train_loss),float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_loss_metric.reset_states()
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in val_dataset:
        val_output, val_preds = model(x_batch_val, training=False)
        # Update val metrics
        val_loss_metric.update_state(y_batch_val, val_output)
        val_acc_metric.update_state(y_batch_val, val_preds)
    
    val_loss = val_loss_metric.result()
    val_loss_metric.reset_states()
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()


    train_loss_results.append(train_loss)
    train_accuracy_results.append(train_acc)
    val_loss_results.append(val_loss)
    val_accuracy_results.append(val_acc)

    print("Validation loss over epoch: %.4f, acc: %.4f" % (float(val_loss),float(val_acc),))
    print("Time taken: %.2fs" % (time.time() - start_time))




Start of epoch 0
Training loss over epoch: 12.4669, acc 0.0263
Validation loss over epoch: 26.9991, acc: 0.0082
Time taken: 144.53s

Start of epoch 1
Training loss over epoch: 10.7696, acc 0.0446
Validation loss over epoch: 25.3617, acc: 0.0095
Time taken: 135.72s

Start of epoch 2
Training loss over epoch: 9.8108, acc 0.0480
Validation loss over epoch: 16.5190, acc: 0.0121
Time taken: 135.81s

Start of epoch 3
Training loss over epoch: 9.3647, acc 0.0570
Validation loss over epoch: 14.6823, acc: 0.0212
Time taken: 135.67s

Start of epoch 4
Training loss over epoch: 8.7647, acc 0.0628
Validation loss over epoch: 12.2538, acc: 0.0285
Time taken: 136.36s

Start of epoch 5
Training loss over epoch: 8.6993, acc 0.0659
Validation loss over epoch: 11.3702, acc: 0.0587
Time taken: 136.25s

Start of epoch 6
Training loss over epoch: 8.0390, acc 0.0737
Validation loss over epoch: 10.1299, acc: 0.0622
Time taken: 136.62s

Start of epoch 7
Training loss over epoch: 7.9957, acc 0.0759
Validation 

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(2, sharex=True, figsize=(12, 8))
fig.suptitle('Training Metrics')

axes[0].set_ylabel("Loss", fontsize=14)
axes[0].set_xlabel("Epoch", fontsize=14)
axes[0].plot(train_loss_results,label="training")
axes[0].plot(val_loss_results,label="validation")
axes[0].legend()


axes[1].set_ylabel("Accuracy", fontsize=14)
axes[1].set_xlabel("Epoch", fontsize=14)
axes[1].plot(train_accuracy_results,label="training")
axes[1].plot(val_accuracy_results,label="validation")
axes[1].legend()

plt.show()

In [ ]:
model.save_weights("model.h5")